In [1]:
!cp /kaggle/input/datasets-wheel/datasets-2.14.4-py3-none-any.whl /kaggle/working
!pip install  /kaggle/working/datasets-2.14.4-py3-none-any.whl



Processing ./datasets-2.14.4-py3-none-any.whl
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [2]:
!cp -r /kaggle/input/stem-wiki-cohere-no-emb /kaggle/working
!cp -r /kaggle/input/all-paraphs-parsed-expanded /kaggle/working/
!cp -r /kaggle/input/new-contexts-3/context_paragraphs_204154.hf /kaggle/working/

In [3]:
from __future__ import annotations
import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm

from collections.abc import Iterable


import matplotlib.pyplot as plt

import torch
import ctypes
libc = ctypes.CDLL("libc.so.6")
from dataclasses import dataclass
from typing import Optional, Union
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings("ignore")



from datasets import load_dataset, load_from_disk
from datasets import concatenate_datasets
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import gc
from transformers import LongformerTokenizer, LongformerForMultipleChoice
import transformers
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import unicodedata
import gc
import pickle
import os


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
def SplitList(mylist, chunk_size):
    return [mylist[offs:offs+chunk_size] for offs in range(0, len(mylist), chunk_size)]

def get_relevant_documents_parsed(df_valid):
    df_chunk_size=600
    paraphs_parsed_dataset = load_from_disk("/kaggle/working/all-paraphs-parsed-expanded")
    my_context = load_from_disk("/kaggle/working/context_paragraphs_204154.hf")
    print(len(paraphs_parsed_dataset),len( my_context ))
    paraphs_parsed_dataset =concatenate_datasets([paraphs_parsed_dataset , my_context])
    print(len(  paraphs_parsed_dataset))
    modified_texts = paraphs_parsed_dataset.map(lambda example:
                                             {'temp_text':
                                              f"{example['title']} {example['section']} {example['text']}".replace('\n'," ")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         paraphs_parsed_dataset[idx.item()]["title"],
                         paraphs_parsed_dataset[idx.item()]["text"],
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles



def get_relevant_documents(df_valid):
    df_chunk_size=800
    
    cohere_dataset_filtered = load_from_disk("/kaggle/working/stem-wiki-cohere-no-emb")
    modified_texts = cohere_dataset_filtered.map(lambda example:
                                             {'temp_text':
                                              unicodedata.normalize("NFKD", f"{example['title']} {example['text']}").replace('"',"")},
                                             num_proc=2)["temp_text"]
    
    all_articles_indices = []
    all_articles_values = []
    for idx in tqdm(range(0, df_valid.shape[0], df_chunk_size)):
        df_valid_ = df_valid.iloc[idx: idx+df_chunk_size]
    
        articles_indices, merged_top_scores = retrieval(df_valid_, modified_texts)
        all_articles_indices.append(articles_indices)
        all_articles_values.append(merged_top_scores)
        
    article_indices_array =  np.concatenate(all_articles_indices, axis=0)
    articles_values_array = np.concatenate(all_articles_values, axis=0).reshape(-1)
    
    top_per_query = article_indices_array.shape[1]
    articles_flatten = [(
                         articles_values_array[index],
                         cohere_dataset_filtered[idx.item()]["title"],
                         unicodedata.normalize("NFKD", cohere_dataset_filtered[idx.item()]["text"]),
                        )
                        for index,idx in enumerate(article_indices_array.reshape(-1))]
    retrieved_articles = SplitList(articles_flatten, top_per_query)
    return retrieved_articles



def retrieval(df_valid, modified_texts):
    
    corpus_df_valid = df_valid.apply(lambda row:
                                     f'{row["prompt"]}\n{row["prompt"]}\n{row["prompt"]}\n{row["A"]}\n{row["B"]}\n{row["C"]}\n{row["D"]}\n{row["E"]}',
                                     axis=1).values
    vectorizer1 = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b",
                                 stop_words=stop_words,sublinear_tf=True)
    vectorizer1.fit(corpus_df_valid)
    vocab_df_valid = vectorizer1.get_feature_names_out()
    vectorizer = TfidfVectorizer(ngram_range=(1,2),
                                 token_pattern=r"(?u)\b[\w/.-]+\b",
                                 stop_words=stop_words,
                                 vocabulary=vocab_df_valid,sublinear_tf=True)
    vectorizer.fit(modified_texts[:6_000_000])
    corpus_tf_idf = vectorizer.transform(corpus_df_valid)
    
    print(f"length of vectorizer vocab is {len(vectorizer.get_feature_names_out())}")

    chunk_size = 100000
    top_per_chunk = 10
    top_per_query = 8

    all_chunk_top_indices = []
    all_chunk_top_values = []

    for idx in tqdm(range(0, len(modified_texts), chunk_size)):
        wiki_vectors = vectorizer.transform(modified_texts[idx: idx+chunk_size])
        temp_scores = (corpus_tf_idf * wiki_vectors.T).toarray()
        chunk_top_indices = temp_scores.argpartition(-top_per_chunk, axis=1)[:, -top_per_chunk:]
        chunk_top_values = temp_scores[np.arange(temp_scores.shape[0])[:, np.newaxis], chunk_top_indices]

        all_chunk_top_indices.append(chunk_top_indices + idx)
        all_chunk_top_values.append(chunk_top_values)

    top_indices_array = np.concatenate(all_chunk_top_indices, axis=1)
    top_values_array = np.concatenate(all_chunk_top_values, axis=1)
    
    merged_top_scores = np.sort(top_values_array, axis=1)[:,-top_per_query:]
    merged_top_indices = top_values_array.argsort(axis=1)[:,-top_per_query:]
    articles_indices = top_indices_array[np.arange(top_indices_array.shape[0])[:, np.newaxis], merged_top_indices]
    
    return articles_indices, merged_top_scores


def prepare_answering_input(
        tokenizer, 
        question,  
        options,   
        context,   
        max_seq_length=4096,
    ):
    c_plus_q   = context + ' ' + tokenizer.bos_token + ' ' + question
    c_plus_q_4 = [c_plus_q] * len(options)
    tokenized_examples = tokenizer(
        c_plus_q_4, options,
        max_length=max_seq_length,
        padding="longest",
        truncation=False,
        return_tensors="pt",
    )
    input_ids = tokenized_examples['input_ids'].unsqueeze(0)
    attention_mask = tokenized_examples['attention_mask'].unsqueeze(0)
    example_encoded = {
        "input_ids": input_ids.to(model.device.index),
        "attention_mask": attention_mask.to(model.device.index),
    }
    return example_encoded
stop_words = ['each', 'you', 'the', 'use', 'used',
                  'where', 'themselves', 'nor', "it's", 'how', "don't", 'just', 'your',
                  'about', 'himself', 'with', "weren't", 'hers', "wouldn't", 'more', 'its', 'were',
                  'his', 'their', 'then', 'been', 'myself', 're', 'not',
                  'ours', 'will', 'needn', 'which', 'here', 'hadn', 'it', 'our', 'there', 'than',
                  'most', "couldn't", 'both', 'some', 'for', 'up', 'couldn', "that'll",
                  "she's", 'over', 'this', 'now', 'until', 'these', 'few', 'haven',
                  'of', 'wouldn', 'into', 'too', 'to', 'very', 'shan', 'before', 'the', 'they',
                  'between', "doesn't", 'are', 'was', 'out', 'we', 'me',
                  'after', 'has', "isn't", 'have', 'such', 'should', 'yourselves', 'or', 'during', 'herself',
                  'doing', 'in', "shouldn't", "won't", 'when', 'do', 'through', 'she',
                  'having', 'him', "haven't", 'against', 'itself', 'that',
                  'did', 'theirs', 'can', 'those',
                  'own', 'so', 'and', 'who', "you've", 'yourself', 'her', 'he', 'only',
                  'what', 'ourselves', 'again', 'had', "you'd", 'is', 'other',
                  'why', 'while', 'from', 'them', 'if', 'above', 'does', 'whom',
                  'yours', 'but', 'being', "wasn't", 'be']

In [5]:

df_valid=pd.read_csv("/kaggle/input/kaggle-llm-science-exam/test.csv")
retrieved_articles_parsed = get_relevant_documents_parsed(df_valid)
gc.collect()
data_=retrieved_articles_parsed 
contexts=np.array([ "\n".join([data_[i][e][2] for e in range(len(data_[0])    )])  for i in range(len(data_)) ])
df_valid["context"]=contexts

2101279 3294990
5396269


Map (num_proc=2):   0%|          | 0/5396269 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

length of vectorizer vocab is 11077



100%|██████████| 1/1 [15:39<00:00, 939.28s/it]


In [6]:
MAX_INPUT = 2500
def competition_score(y_true, y_pred):

        """
        Obtaining score for the model
        """

        ap_at_3 = 0.0
        for i in range(len(y_true)):
            if y_true[i]==  y_pred[i][0]:
                ap_at_3 += 1
            elif y_true[i]==  y_pred[i][1]:
                ap_at_3 += 1/2
            elif y_true[i]==  y_pred[i][2]:
                ap_at_3 += 1/3


        map3 = ap_at_3 / len(y_true)
        return map3

def predictions_to_map_output(predictions):
        sorted_answer_indices = np.argsort(-predictions) # Sortting indices in descending order
        top_answer_indices = sorted_answer_indices[:,:3] # Taking the first three indices for each row
        top_answers = np.vectorize(index_to_option.get)(top_answer_indices) # Transforming indices to options - i.e., 0 --> A
        return  top_answers
def compute_metrics(eval_preds):
        logits, labels = eval_preds
        y_pred = predictions_to_map_output(logits)
        y_true = [index_to_option[label] for label in labels]
        print( y_true)
        return {'Map@3': np.round(competition_score(y_true, y_pred), 3)}








def preprocess_wrapper(tokenizer,MAX_INPUT=MAX_INPUT):
    def preprocess(example):

        first_sentence = [example['prompt']] * 5 # Repeating the same question 5 times
        second_sentence = [] # Creating list of possible answers
        for option in options:
            second_sentence.append(example[option])

        # The tokenizer converts the question and answers into 'tokens'.
        # 'tokens' are simply a sequence of integers in which each specific integer corresponds to
        # a word or subword that BERT is capable of comprehending
        tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)

        # Indexing label - A,B,C,D or E - to either 0, 1, 2, 3, 4, or 5
        tokenized_example['label'] = option_to_index[example['answer']]

        # tokenized_example returns:
            # input_ids --> List of lists represents the tokens
            # token_type_ids --> A list of lists indicating whether each token belongs to the 1st or 2nd sentence
            # attention_mask --> List of list where each inner list is either 1 or 0. 1 are not padding tokens, 0 are padding tokens
            # label --> The index for the correct option
        return tokenized_example

    def preprocess_with_context(example):

        first_sentence = [ "[CLS] " + example['context'] ] * 5
        second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']

        #tokenized_example = tokenizer(first_sentence, second_sentences, truncation=True)
        tokenized_example = tokenizer(first_sentence, second_sentences, truncation='only_first',
                                        max_length=MAX_INPUT, add_special_tokens=False)
        tokenized_example['label'] = option_to_index[example['answer']]

        return tokenized_example

    return preprocess_with_context




@dataclass
class DataCollatorForMultipleChoice:
    '''
    This class is designed to handle the formatting and batching the data for mutiple-choice tasks
    '''

    # The tokenizer to be used for tokenizing the data
    tokenizer: PreTrainedTokenizerBase

    # The strategy to be used for padding the data
    padding: Union[bool, str, PaddingStrategy] = True

    # The maximum length for any input sequence
    max_length: Optional[int] = None

    # If provided, pad the sequences to a multiple of this value
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        # Finding the correct label key in the features
        label_name = "label" if 'label' in features[0].keys() else 'labels'

        # Extracting the labels and removing them from features
        labels = [feature.pop(label_name) for feature in features]

        # Obtaining batch size
        batch_size = len(features)

        # Obtaining number of choices
        num_choices = len(features[0]['input_ids'])

        # Reestructuring features so each question-choice pair becomes a separate example
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        # Padding all sequences to the same length
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )

        # Reshaping the batch back into the original format
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}

        # Adding the labels back into the batch as a tensor
        batch['labels'] = torch.tensor(labels,
                                    dtype=torch.int64)

        # Returning the batch
        return batch




test_df = df_valid.copy()
test_df.index = list(range(len(test_df)))
test_df['id'] = list(range(len(test_df)))
#test_df["prompt"] = test_df["context"].apply(lambda x: x[:2000]) + " #### " +  test_df["prompt"]
test_df['answer'] = 'A'
test_ds=Dataset.from_pandas(test_df)



r_cols= [e for e in ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer', '__index_level_0__','context','source','id']  if e in test_ds.features]



model_dir = "/kaggle/input/llm-science-run-context-2"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()


options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}




tokenized_test_dataset= test_ds.map(preprocess_wrapper(tokenizer,MAX_INPUT=MAX_INPUT), batched=False,
                            remove_columns=r_cols)
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)


def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / e_x.sum(axis=-1, keepdims = True)    


test_predictions1 = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictions1.append(outputs.logits.cpu().detach())

test_predictions1 = torch.cat(test_predictions1)
test_predictions1 = test_predictions1.numpy()


options = 'ABCDE'
indices = list(range(5)) # Indexing 0, 1, 2, 3, 4 for each option


option_to_index = {option: index for option, index in zip(options, indices)} # Converting options to indices '''A to 0'''
index_to_option = {index: option for option, index in zip(options, indices)} # Converting indices to options '''0 to A'''





model_dir = "/kaggle/input/big-models/checkpoint-23000"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()

test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)

test_predictions2 = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictions2.append(outputs.logits.cpu().detach())

test_predictions2 = torch.cat(test_predictions2)
test_predictions2 = test_predictions2.numpy()


model_dir = "/kaggle/input/change-validation/checkpoint18576/checkpoints/checkpoint-18576"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
model.eval()



test_predictions3 = []
for batch in test_dataloader:
    for k in batch.keys():
        batch[k] = batch[k].cuda()
    with torch.no_grad():
        outputs = model(**batch)
    test_predictions3.append(outputs.logits.cpu().detach())

test_predictions3 = torch.cat(test_predictions3)
test_predictions3 = test_predictions3.numpy()


test_predictions = 0.2*softmax (test_predictions1) + 0.6*softmax(test_predictions2) + 0.2*softmax(test_predictions3) 

predictions_as_ids = np.argsort(-test_predictions, 1)

predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]

test_df['prediction'] = [
    ' '.join(row) for row in predictions_as_answer_letters[:, :3]
]


submission = test_df[['id', 'prediction']]
submission.to_csv('submission.csv', index=False)


df = pd.read_csv('/kaggle/input/60k-data-with-context-v2/train_with_context2.csv') # Loading data
test_df["answer"] = df["answer"]


print(competition_score(test_df["answer"], predictions_as_answer_letters))






Map:   0%|          | 0/200 [00:00<?, ? examples/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


0.9925


In [7]:

for pred in [test_predictions1,test_predictions2,test_predictions3]:
    predictions_as_ids = np.argsort(-pred, 1)

    predictions_as_answer_letters = np.array(list('ABCDE'))[predictions_as_ids]


    df = pd.read_csv('/kaggle/input/60k-data-with-context-v2/train_with_context2.csv') # Loading data
    test_df["answer"] = df["answer"]


    print(competition_score(test_df["answer"], predictions_as_answer_letters))

0.99
0.9875
0.9825


Thanks to https://www.kaggle.com/code/mewmlelswm/interesting-combination-of-average-experiments/notebook

          https://www.kaggle.com/code/cdeotte/how-to-train-open-book-model-part-2